In [1]:
import os 
cwd = os.getcwd()

os.chdir('../')
cwd = os.getcwd()
print(cwd)
 
os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

import pandas as pd 
import numpy as np 
import seaborn as sns

import pickle 
import logging


from config.config import SQLQuery
querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/RS2_refit
/Users/shashankgupta/Documents/code/git_project/RS2_refit/Code


In [2]:
base_df = pd.read_csv('/Users/shashankgupta/Documents/code/git_project/RS2_refit/Data/base_plaid_coverage.csv')
base_df.shape

(710, 4)

In [3]:
base_df.rename(columns={'BUSINESS_ID':'business_id'},inplace=True)

In [4]:
base_df.head()

,LENDING_BUSINESS_ID,business_id,Ever15plus,Source
0,b88bad24-8711-444a-98e1-52047e87f6fc,fdbeb997-87d8-411e-a068-9a2297324910,0,GA
1,987da0d5-f838-45a9-8bb6-d19a64e8888c,215ca059-aac1-4c09-a6db-9e2a143ce7c5,0,GA
2,166d62fb-70ed-4214-8a75-1223dfbbd06e,252c9a3f-338f-43fc-893f-87e4c8184e8c,0,GA
3,8105f405-a33a-434e-9f0d-0c0fed3b0889,02d133cb-e8eb-449d-9b7d-d0501058bc61,0,GA
4,13fe3447-98d9-4996-92e5-fffa8e63429b,e98ced58-b53b-44c0-8cfa-2dd9b43f95be,0,GA


In [5]:
querySno = SQLQuery('snowflake')
txn_df = querySno(r""" 
WITH past_6_mo_txn as (
SELECT transaction_id, BUSINESS_ID , amount, TYPE, created_at as TRANSACTION_DATE, sum(amount) over(partition by business_id order by created_at) running_balance,
              CASE WHEN TYPE = 'credit' and DESCRIPTION  ILIKE ANY(
	                    '%%AMAZON%%'
	                    , '%%AMZN%%'
	                    , '%%STRIPE%%'
	                    , '%%SQUARE INC%%'
	                    , '%%SHOPIFY%%'
	                    , '%%SHOPPAY%%'
	                    )
                    AND NOT(
	                    description ILIKE ANY(
	                        '%%LYFT'
	                        , '%%OfferUp%%'
	                        , '%%Gumroad%%'
	                        , '%%FB%%Fundrai%%'
	                        , '%%Verify%%'
	                        , '%%CASH%%'
	                        , '%%PAYROLL%%'
	                        , '%%VRFY%%'
	                        , '%%CAPITAL%%'
	                        , '%%REFUND%%'
	                    ) ) THEN 1 ELSE 0 END as PD_DEPOSIT_TXN,
                CASE WHEN TYPE ='debit' AND lower(description)  ILIKE ANY('%%ach withdrawal%%', '%%ach deposit%%', '%%withdrawal ach%%', '%%ach payment%%','%%ach pmt%%', '%%ach debit%%', '%%ach credit%%') THEN 1 ELSE 0 END AS ach_debit_txn,
                CASE WHEN type = 'credit'  AND lower(description) ILIKE ANY('%%ach withdrawal%%', '%%ach deposit%%', '%%withdrawal ach%%', '%%ach payment%%','%%ach pmt%%', '%%ach debit%%', '%%ach credit%%') THEN 1 ELSE 0 END AS ach_credit_txn,
                -- CASE WHEN medium = 'POS Withdrawal' THEN 1 ELSE 0 END AS card_withdrawal_txn,
                CASE WHEN TYPE = 'credit' AND lower(description)='%%mobile check deposit%%' THEN 1 ELSE 0 END AS mrdc_credit_txn,
                datediff(day, TRANSACTION_DATE, '2023-06-10') AS daySinceLoanTaken
-- FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS"  
from "METABASE_DB"."TRANSACTION"."PLAID_TRANSACTIONS"
where daySinceLoanTaken <=180 AND daySinceLoanTaken > 0
),
cal_dates_1 AS (
			SELECT DATEADD(DAY, SEQ4(), '2022-12-01') AS CAL_DATE
			 FROM TABLE(GENERATOR(ROWCOUNT=>365)) 
			 ORDER BY CAL_DATE DESC),
biz_dates_1 AS (
			SELECT a.BUSINESS_ID, MIN(a.created_at) AS FIRST_TXN_DATE
			from "METABASE_DB"."TRANSACTION"."PLAID_TRANSACTIONS"  a 
			GROUP BY a.BUSINESS_ID),
PILOT_BIZ_DATES_TEMP AS (
		SELECT *
		FROM  cal_dates_1 A LEFT JOIN
		biz_dates_1 B ON
		date(A.CAL_DATE) >= date(B.FIRST_TXN_DATE)
		ORDER BY CAL_DATE ASC)
,PILOT_BIZ_TXN_TEMP AS (
		SELECT   Business_id ,TRANSACTION_DATE, running_balance
		FROM
		        (SELECT *, RANK () OVER (PARTITION BY BUSINESS_ID,TRANSACTION_DATE ORDER BY TRANSACTION_DATE DESC) RANKS
		         FROM past_6_mo_txn
                 where 1=1
		         )
		WHERE RANKS=1
		order by Business_id, TRANSACTION_DATE)
,PILOT_DAILY_BALANCES_1  AS (
		SELECT BUSINESS_ID,FIRST_TXN_DATE, CAL_DATE,TRANSACTION_DATE,RUNNING_BALANCE,
		first_value(RUNNING_BALANCE) OVER (PARTITION BY BUSINESS_ID,GROUPER ORDER BY cal_date asc, TRANSACTION_DATE desc, RUNNING_BALANCE asc nulls last) as RUNNING_BALANCE_2
		FROM (
		SELECT A.BUSINESS_ID,A.CAL_DATE,A.FIRST_TXN_DATE,
		        B.TRANSACTION_DATE,B.RUNNING_BALANCE,
		         COUNT(running_balance) OVER (PARTITION BY a.BUSINESS_ID ORDER BY cal_date asc) as grouper
		        FROM PILOT_BIZ_DATES_TEMP A LEFT JOIN
		        PILOT_BIZ_TXN_TEMP B ON
		        A.BUSINESS_ID=B.BUSINESS_ID AND
		        date(A.CAL_DATE)= date(B.TRANSACTION_DATE)
		        ORDER BY A.CAL_DATE ASC)
		ORDER BY BUSINESS_ID,CAL_DATE)
,final_daily_running_balance AS (
		 SELECT distinct a.business_id, datediff(day, a.cal_date, current_timestamp) as daySinceLoanTaken, cal_date, running_balance_2 AS running_balance_daily
		           FROM  PILOT_DAILY_BALANCES_1  a   
		          WHERE daySinceLoanTaken > 0 AND daySinceLoanTaken <= 180
		ORDER BY a.business_id, daySinceLoanTaken)
,oneMonth_rb AS (	
				SELECT business_id, min(cal_date) as min_txn_date_1mo_rb, max(cal_date) as max_txn_date_1mo_rb,
				               sum(CASE WHEN running_balance_daily < 0 THEN  1 ELSE 0 end) AS od_count_1m, 
				               sum(CASE WHEN running_balance_daily = 0 THEN  1 ELSE 0 end) AS zero_balance_count_1m,
				               AVG(running_balance_daily) as Avg_running_balance_1M,
  							   Median(running_balance_daily) as Median_running_balance_1M,
							   STDDEV(running_balance_daily) as STDDEV_running_balance_1M
				FROM final_daily_running_balance
				WHERE daySinceLoanTaken <=30
				GROUP BY business_id),
threeMonths_rb AS (	
				SELECT business_id, min(cal_date) as min_txn_date_3mo_rb, max(cal_date) as max_txn_date_3mo_rb,
				               sum(CASE WHEN running_balance_daily < 0 THEN  1 ELSE 0 end) AS od_count_3m, 
				               sum(CASE WHEN running_balance_daily = 0 THEN  1 ELSE 0 end) AS zero_balance_count_3m,
				               AVG(running_balance_daily) as Avg_running_balance_3M,
  							   Median(running_balance_daily) as Median_running_balance_3M,
							   STDDEV(running_balance_daily) as STDDEV_running_balance_3M
				FROM final_daily_running_balance
				WHERE daySinceLoanTaken <=90
				GROUP BY business_id),
 sixMonths_rb AS (	
				SELECT business_id, min(cal_date) as min_txn_date_6mo_rb, max(cal_date) as max_txn_date_6mo_rb,
				               sum(CASE WHEN running_balance_daily < 0 THEN  1 ELSE 0 end) AS od_count_6m, 
				               sum(CASE WHEN running_balance_daily = 0 THEN  1 ELSE 0 end) AS zero_balance_count_6m,
				               AVG(running_balance_daily) as Avg_running_balance_6M,
  							   Median(running_balance_daily) as Median_running_balance_6M,
							   STDDEV(running_balance_daily) as STDDEV_running_balance_6M
				FROM final_daily_running_balance
				WHERE daySinceLoanTaken <=180
				GROUP BY business_id),
aggregate_1_3_6_rb as (
SELECT business_id, zero_balance_count_1m, od_count_3m, COALESCE(Median_running_balance_6M, 0) as Median_running_balance_6M, 
       max_txn_date_6mo_rb, min_txn_date_6mo_rb, max_txn_date_3mo_rb, min_txn_date_3mo_rb, max_txn_date_1mo_rb, min_txn_date_1mo_rb,
       STDDEV_RUNNING_BALANCE_6M
FROM
sixMonths_rb AS a
FULL OUTER JOIN 
threeMonths_rb AS b
using (BUSINESS_ID)
FULL OUTER JOIN 
oneMonth_rb AS c
USING (BUSINESS_ID)
),
past_3_mo_txn AS (
SELECT * FROM past_6_mo_txn WHERE daySinceLoanTaken <= 90
),
past_1mo_txn as(
SELECT * FROM PAST_6_MO_txn WHERE daySinceLoanTaken <= 30
),
past_6_mo_agg AS (
SELECT BUSINESS_ID, min(transaction_date) as min_txn_date_6mo, max(transaction_date) as max_txn_date_6mo,
			  sum(CASE WHEN ach_credit_txn=1 THEN abs(amount) ELSE 0 end) AS ach_credit_amt_180,
			  sum(CASE WHEN ach_debit_txn=1 THEN abs(amount) ELSE 0 end) AS ach_debit_amt_180,
			  sum(CASE WHEN ach_credit_txn=1 AND abs(amount) > 100 THEN 1 ELSE 0 end) AS distinct_ach_c_txns_100_6m,
			  COALESCE(median(CASE WHEN ach_debit_txn=1 THEN abs(amount) ELSE NULL end), 0) AS median_amount_ach_d_6m,
			  COALESCE(sum(CASE WHEN ach_debit_txn=1 THEN abs(amount) ELSE 0 end) / NULLIF(sum(CASE WHEN ach_credit_txn=1 THEN abs(amount) ELSE 0 end), 0), 0) AS ratio_ach_amt_debit_credit_6m,
              COALESCE(stddev(CASE WHEN ach_credit_txn=1 THEN abs(amount) ELSE null end), 0) AS stddev_amount_ach_c_6m,
              COALESCE(sum(ach_debit_txn) / NULLIF(sum(ach_credit_txn), 0), 0) AS ratio_ach_freq_debit_credit_6m,
              COALESCE(sum(CASE WHEN TYPE='debit' THEN 1 ELSE 0 end) /  NULLIF(sum(CASE WHEN TYPE='credit' THEN 1 ELSE 0 end), 0), 0) as ratio_total_freq_debit_credit_6m
FROM past_6_mo_txn GROUP BY BUSINESS_ID 
),
past_3_mo_agg AS (
SELECT BUSINESS_ID, min(transaction_date) as min_txn_date_3mo, max(transaction_date) as max_txn_date_3mo,
              sum(CASE WHEN ach_credit_txn=1 THEN abs(amount) ELSE 0 end) AS ach_credit_amt_90,
			  sum(CASE WHEN ach_debit_txn=1 THEN abs(amount) ELSE 0 end) AS ach_debit_amt_90,
              COALESCE(sum(CASE WHEN TYPE='debit' THEN abs(amount) ELSE 0 end) / NULLIF(sum(CASE WHEN TYPE='credit' THEN abs(amount) ELSE 0 end), 0), 0) AS ratio_debit_credit_3m,
              COALESCE(median(CASE WHEN TYPE='debit' THEN abs(amount) ELSE null end), 0) as median_amount_debited_3m,
              COALESCE(median(CASE WHEN TYPE='credit' THEN abs(amount) ELSE null end), 0) as median_amount_credited_3m,
              COALESCE(sum(ach_debit_txn), 0) as distinct_ach_d_txns_3m
FROM past_3_mo_txn GROUP BY BUSINESS_ID 
),
past_1_mo_agg AS (
SELECT BUSINESS_ID, min(transaction_date) as min_txn_date_1mo, max(transaction_date) as max_txn_date_1mo,
               STDDEV(CASE WHEN ach_credit_txn=1 THEN abs(AMOUNT) ELSE null end) AS stddev_amount_ach_c_1m,
               sum(mrdc_credit_txn) AS distinct_mrdc_txns_1m,
               COALESCE(sum(ach_debit_txn) / NULLIF(sum(ach_credit_txn), 0), 0) as ratio_ach_freq_debit_credit_1m,
               COALESCE(sum(CASE WHEN TYPE='debit' THEN abs(amount) ELSE 0 end) / NULLIF(sum(CASE WHEN TYPE='credit' THEN abs(amount) ELSE 0 end), 0), 0) AS ratio_debit_credit_1m,
               COALESCE(sum(CASE WHEN TYPE='debit' THEN abs(amount) ELSE null end), 0) as stddev_amount_debited_1m,
               COALESCE(sum(ach_debit_txn), 0) as distinct_ach_d_txns_1m,
               COALESCE(sum(CASE WHEN ach_debit_txn=1 THEN abs(amount) ELSE 0 end) / NULLIF(sum(CASE WHEN ach_credit_txn=1 THEN abs(amount) ELSE 0 end), 0), 0) AS ratio_ach_amt_debit_credit_1m
               
               FROM past_1mo_txn
               GROUP BY business_id
),

business_info AS (
SELECT b.BUSINESS_ID, NUMBER_OF_EMPLOYEES::int as NUMBER_OF_EMPLOYEES, 
               datediff(MONTH, to_date(DATE_OF_ESTABLISHMENT, 'YYYY-MM'), current_timestamp) AS business_age_months, 
               datediff(month, ACCOUNT_create_date, current_timestamp) AS months_on_book
FROM  PROD_DB.DATA.businesses b 
      LEFT JOIN PROD_DB.DATA.APPLICATIONS c ON c.application_id = b.application_id
)
		
SELECT BUSINESS_ID, od_count_3m, zero_balance_count_1m, 
			   COALESCE(ach_credit_amt_90/ NULLIF(ach_credit_amt_180, 0), 0) AS ratio_ach_credit_amt_90_180,
			   COALESCE(ach_debit_amt_90/ NULLIF(ach_debit_amt_180, 0), 0) AS ratio_ach_debit_amt_90_180, 
			   COALESCE(stddev_amount_ach_c_1m, 0) as stddev_amount_ach_c_1m, COALESCE(distinct_ach_c_txns_100_6m, 0) as distinct_ach_c_txns_100_6m, 
			   COALESCE(median_amount_ach_d_6m, 0) as median_amount_ach_d_6m, 
			   COALESCE(distinct_mrdc_txns_1m, 0) as distinct_mrdc_txns_1m,
			   COALESCE(ratio_ach_amt_debit_credit_6m, 0) as ratio_ach_amt_debit_credit_6m, COALESCE(ratio_ach_freq_debit_credit_1m, 0) as ratio_ach_freq_debit_credit_1m, 
			   COALESCE(ratio_debit_credit_1m, 0) as ratio_debit_credit_1m, COALESCE(ratio_debit_credit_3m, 0) as ratio_debit_credit_3m, 
			   median_running_balance_6m, median_amount_debited_3m, stddev_amount_debited_1m, stddev_amount_ach_c_6m, distinct_ach_d_txns_1m,
               ratio_ach_amt_debit_credit_1m, ratio_ach_freq_debit_credit_6m, 
               median_amount_credited_3m, distinct_ach_d_txns_3m,
               ratio_total_freq_debit_credit_6m, STDDEV_RUNNING_BALANCE_6M, 
               months_on_book, business_age_months, number_of_employees
FROM aggregate_1_3_6_rb 
	LEFT JOIN  past_6_mo_agg using(business_id)
	LEFT JOIN past_3_mo_agg using(business_id)
	LEFT JOIN past_1_mo_agg using(business_id)
    LEFT JOIN business_info using(business_id) 
""")

In [6]:
txn_df.shape

(107989, 27)

In [7]:
txn_df.head()

,business_id,od_count_3m,zero_balance_count_1m,ratio_ach_credit_amt_90_180,ratio_ach_debit_amt_90_180,stddev_amount_ach_c_1m,distinct_ach_c_txns_100_6m,median_amount_ach_d_6m,distinct_mrdc_txns_1m,ratio_ach_amt_debit_credit_6m,...,distinct_ach_d_txns_1m,ratio_ach_amt_debit_credit_1m,ratio_ach_freq_debit_credit_6m,median_amount_credited_3m,distinct_ach_d_txns_3m,ratio_total_freq_debit_credit_6m,stddev_running_balance_6m,months_on_book,business_age_months,number_of_employees
0,d51ce771-f6c7-488c-81c8-9b2fef3b5fe6,35,0,0.0,0.0,0.0,0,0.00,0,0.000000,...,0.0,0.0,0.0,169.685,0.0,5.972851,582.836961,8,8.0,1.0
1,543a5fa6-bbf3-44e5-926e-b3654a4a36fb,63,0,0.0,0.0,0.0,0,0.00,0,0.000000,...,0.0,0.0,0.0,250.000,0.0,4.833333,997.477405,20,23.0,1.0
2,eb383785-0e4d-484e-8376-8453ae423ba7,0,0,0.0,0.0,0.0,0,0.00,0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,43.0,2.0
3,a8c26584-349f-4b9d-95bb-9c798a49ed00,18,0,0.0,0.0,0.0,0,0.00,0,0.000000,...,0.0,0.0,0.0,75.000,0.0,6.970588,156.044196,4,NaN,1.0
4,39006d6f-3753-49be-902c-b0c7b14db072,90,0,0.0,0.0,0.0,32,101.41,0,0.060458,...,0.0,0.0,0.5,200.000,0.0,5.175719,5464.581067,4,NaN,2.0


In [8]:
df_raw = base_df.merge(txn_df, how='left', left_on='business_id',right_on='business_id')

In [9]:
df_raw.shape

(710, 30)

In [10]:
final_cols = ['business_id','od_count_3m', 'zero_balance_count_1m',
       'ratio_ach_credit_amt_90_180', 'ratio_ach_debit_amt_90_180',
       'stddev_amount_ach_c_1m', 'distinct_ach_c_txns_100_6m',
       'distinct_mrdc_txns_1m', 'ratio_debit_credit_1m',
       'ratio_debit_credit_3m', 'median_running_balance_6m','Ever15plus']

In [11]:
df = df_raw[final_cols]

In [14]:
df.describe()

,od_count_3m,zero_balance_count_1m,ratio_ach_credit_amt_90_180,ratio_ach_debit_amt_90_180,stddev_amount_ach_c_1m,distinct_ach_c_txns_100_6m,distinct_mrdc_txns_1m,ratio_debit_credit_1m,ratio_debit_credit_3m,median_running_balance_6m,Ever15plus
count,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000,678.0,678.000000,678.000000,678.000000,710.000000
mean,41.603245,0.057522,0.066736,0.173129,96.801188,4.507375,0.0,24.288397,1.132973,1604.165450,0.229577
std,35.316385,1.192515,0.224531,0.310700,653.005352,26.823001,0.0,548.702040,3.147476,25852.628125,0.420858
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-241733.450000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.759249,0.918608,-1153.827500,0.000000
50%,40.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.992756,0.997264,0.000000,0.000000
75%,77.000000,0.000000,0.000000,0.208006,0.000000,0.000000,0.0,1.123496,1.041602,1667.632500,0.000000
max,90.000000,30.000000,1.000000,1.000000,7898.392618,432.000000,0.0,14275.000000,77.649939,348697.955000,1.000000


In [16]:
data_params_rs2 = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/RS2_refit/Models/data_params_v2.pkl')
data_params_rs2

,feature,median,lower_limit,upper_limit
3,od_count_3m,0.000000,0.0000,50.480000
6,zero_balance_count_1m,0.000000,0.0000,5.740000
8,ratio_ach_credit_amt_90_180,0.481349,0.0000,1.000000
10,ratio_ach_debit_amt_90_180,0.437795,0.0000,1.000000
64,stddev_amount_ach_c_1m,111.338495,0.0000,6918.901000
71,distinct_ach_c_txns_100_6m,14.000000,0.0000,94.740000
113,distinct_mrdc_txns_1m,0.000000,0.0000,9.220000
133,ratio_debit_credit_1m,1.004027,0.0000,537.812312
134,ratio_debit_credit_3m,1.027857,0.0000,14.564760
137,median_running_balance_6m,743.540000,0.9546,33732.172800


In [15]:
df['Ever15plus'].value_counts()

0    547
1    163
Name: Ever15plus, dtype: int64

In [13]:
# df.to_pickle('/Users/shashankgupta/Documents/code/git_project/RS2_refit/Data/plaid_lending_data.pkl')